In [1]:
!pip install kaggle

In [2]:
!mkdir ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download shivamb/netflix-shows

  0% 0.00/1.18M [00:00<?, ?B/s]
100% 1.18M/1.18M [00:00<00:00, 38.9MB/s]


In [6]:
!unzip netflix-shows.zip

Archive:  netflix-shows.zip
  inflating: netflix_titles.csv      


In [8]:
pip install dash

     |████████████████████████████████| 7.3 MB 14.1 MB/s 
     |████████████████████████████████| 23.9 MB 13 kB/s 
     |████████████████████████████████| 357 kB 42.0 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=82832312a220e3268479ce0cb8501ce7800ca715f82352d22dbf0d845f22af42
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=a4f1ca95d9f1fcb4e64926eb29340113861be3e1988a9a503443c933ee633447
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=e16aa74aac0132a46ab3738c2396a5783f5acec041e3333ccf61d699efe6d5da
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aef

In [4]:
pip install dash_bootstrap_components

     |████████████████████████████████| 197 kB 12.2 MB/s 


In [5]:
import dash
import dash_core_components as dcc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
from textblob import TextBlob
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.io as plt_io



In [6]:
app = dash.Dash(__name__,title="NETFLIX ANALYTICS",external_stylesheets=[dbc.themes.GRID])

In [7]:
server=app.server

In [64]:
dff = pd.read_csv('netflix_titles.csv')

In [65]:
plt_io.templates["custom_dark"] = plt_io.templates["plotly_dark"]
plt_io.templates["custom_dark"]['layout']['paper_bgcolor'] = '#1e1e2e'
plt_io.templates["custom_dark"]['layout']['plot_bgcolor'] = '#1e1e2e'
plt_io.templates['custom_dark']['layout']['yaxis']['gridcolor'] = '#342c47'
plt_io.templates['custom_dark']['layout']['xaxis']['gridcolor'] = '#342c47'

In [69]:
z = dff.groupby(['rating']).size().reset_index(name='counts')
pieChart = px.pie(z, values='counts', names='rating', title='Distribution of Content Ratings on Netflix 🍕🍿🥃😄▶️💻',color_discrete_sequence=px.colors.qualitative.Set3)
pieChart.layout.template = 'custom_dark'

In [70]:
pieChart

In [71]:
dff['director']=dff['director'].fillna('No Director Specified')
filtered_directors=pd.DataFrame()
filtered_directors=dff['director'].str.split(',',expand=True).stack()
filtered_directors=filtered_directors.to_frame()
filtered_directors.columns=['Director']
directors=filtered_directors.groupby(['Director']).size().reset_index(name='Total Content')
directors=directors[directors.Director !='No Director Specified']
directors=directors.sort_values(by=['Total Content'],ascending=False)
directorsTop5=directors.head()
directorsTop5=directorsTop5.sort_values(by=['Total Content'])
fig2=px.bar(directorsTop5,x='Total Content',y='Director',title='Top 5 Directors on Netflix 🎬 📝')
fig2.layout.template = 'custom_dark'

In [72]:
fig2

In [73]:
dff['cast']=dff['cast'].fillna('No Cast Specified')
filtered_cast=pd.DataFrame()
filtered_cast=dff['cast'].str.split(',',expand=True).stack()
filtered_cast=filtered_cast.to_frame()
filtered_cast.columns=['Actor']
actors=filtered_cast.groupby(['Actor']).size().reset_index(name='Total Content')
actors=actors[actors.Actor !='No Cast Specified']
actors=actors.sort_values(by=['Total Content'],ascending=False)
actorsTop5=actors.head()
actorsTop5=actorsTop5.sort_values(by=['Total Content'])
fig3=px.bar(actorsTop5,x='Total Content',y='Actor', title='Top 5 Actors on Netflix 💃')
fig3.layout.template = 'custom_dark'

In [74]:
fig3

In [75]:
df1=dff[['type','release_year']]
df1=df1.rename(columns={"release_year": "Release Year"})
df2=df1.groupby(['Release Year','type']).size().reset_index(name='Total Content')
df2=df2[df2['Release Year']>=2010]
fig4 = px.line(df2, x="Release Year", y="Total Content", color='type',title='Trend of content produced over the years on Netflix 🍕🍿🥃😄▶️💻')
fig4.layout.template = 'custom_dark'

In [76]:
fig4

In [78]:
dfx=dff[['release_year','description']]
dfx=dfx.rename(columns={'release_year':'Release Year'})
for index,row in dfx.iterrows():
    z=row['description']
    testimonial=TextBlob(z)
    p=testimonial.sentiment.polarity
    if p==0:
        sent='Neutral 🔹'
    elif p>0:
        sent='Positive ✔️'
    else:
        sent='Negative ❌'
    dfx.loc[[index,2],'Sentiment']=sent


dfx=dfx.groupby(['Release Year','Sentiment']).size().reset_index(name='Total Content')

dfx=dfx[dfx['Release Year']>=2010]
fig5 = px.bar(dfx, x="Release Year", y="Total Content", color="Sentiment", title="Content Sentiment on Netflix 😃😑😥😡")
fig5.layout.template = 'custom_dark'

In [79]:
fig5